In [1]:
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_origin
import numpy as np
import os

In [11]:
def find_parquet_files(directory):
    """
    Find all Parquet files within a directory and its subdirectories.

    :param directory: The base directory to search.
    :return: A list of paths to Parquet files.
    """
    parquet_files = []
    for root, dirs, files in os.walk(directory):
        print(root,dirs,files)
        for file in files:
            print(file)
            if file.endswith('.parquet'):
                parquet_files.append(os.path.join(root, file))
    return parquet_files

In [3]:
def rasterize_dataset(dataset_path, raster_path, raster_shape, raster_transform):
    """
    Rasterize a single geospatial dataset.

    :param dataset_path: Path to the input geospatial dataset (e.g., a Parquet file).
    :param raster_path: Path where the rasterized output will be saved.
    :param raster_shape: Shape of the output raster (height, width).
    :param raster_transform: Transformation for the output raster.
    """
    # Load the dataset
    df = pd.read_parquet(dataset_path)
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:32161")

    # Rasterize
    shapes = ((geom, 1) for geom in gdf.geometry)
    rasterized = rasterize(shapes, out_shape=raster_shape, transform=raster_transform, fill=0, default_value=1, dtype='uint8')

    # Save the rasterized output
    with rasterio.open(
        raster_path, 'w', 
        driver='GTiff', 
        height=raster_shape[0], 
        width=raster_shape[1], 
        count=1, 
        dtype='uint8', 
        crs='EPSG:32161', 
        transform=raster_transform
    ) as dst:
        dst.write(rasterized, 1)

In [4]:
def combine_rasters(raster_paths, output_path):
    """
    Combine multiple rasterized datasets into a single raster.

    :param raster_paths: List of paths to the rasterized datasets.
    :param output_path: Path where the combined raster will be saved.
    """
    combined_raster = None
    profile = None

    for path in raster_paths:
        with rasterio.open(path) as src:
            raster_data = src.read(1)  # Read the first band
            if combined_raster is None:
                combined_raster = raster_data
                profile = src.profile
            else:
                combined_raster = np.maximum(combined_raster, raster_data)  # Combine using maximum value

    # Modify the profile to accommodate the combined raster
    profile.update(dtype=rasterio.uint8, count=1)

    # Save the combined raster
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(combined_raster, 1)

In [5]:
def raster_to_dataframe(raster_path):
    """
    Convert a raster to a Pandas DataFrame.

    :param raster_path: Path to the raster file.
    :return: DataFrame representation of the raster.
    """
    with rasterio.open(raster_path) as src:
        raster_data = src.read(1)
    return pd.DataFrame(raster_data)

In [13]:
# Find all Parquet files in the specified directory
base_directory = 'data'
parquet_files = find_parquet_files(base_directory)

print(parquet_files)

# Define the raster shape and transformation
raster_shape = (200, 400)  # Example shape based on your grid
raster_transform = from_origin(100000, 300000, 500, 500)  # Example transform

# Paths for the rasterized outputs
raster_paths = [os.path.splitext(p)[0] + '.tif' for p in parquet_files]

# Rasterize each dataset
for dataset_path, raster_path in zip(parquet_files, raster_paths):
    rasterize_dataset(dataset_path, raster_path, raster_shape, raster_transform)

# Combine the rasterized datasets
combined_raster_path = 'combined_raster.tif'
combine_rasters(raster_paths, combined_raster_path)

# Convert the combined raster to a DataFrame
final_df = raster_to_dataframe(combined_raster_path)

[]


AttributeError: 'NoneType' object has no attribute 'update'